# Pytorch与MNIST

## 说明

程序源代码参考：https://github.com/pytorch/examples/blob/master/mnist/main.py

本代码是官方示例的简化分析版本，网络结构和官方版本一致。对于MNSIT的基本分析参考第一节。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## MNIST数据集

MNIST的图像数据是28像素×28像素的灰度图像（1通道），各个像素的取值在0到255之间。每个图像都相应地标有“1”、“2”、“3”等标签

## 设定网络结构

- 第一个__init__是初始化的参数，包含两个卷积层(conv)和两个全连接层(fc)。

```Python
torch.nn.Linear(in_features, out_features, bias=True)
torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
```



> - conv1采用Conv2d层实现，其参数意义为：输入通道为1（灰度图像），输出通道为20（人工设定），卷积核为5×5，步长为1。
>
> - conv2也采用Conv2d层实现，其参数意义为：输入通道为20（同conv1输出），输出通道为50（人工设定），卷积核为5×5，步长为1。
>
> - fc1采用线性变换层实现，输入样本大小为4\*4\*50（50个4\*4的核），输出样本大小为500（人工设定）
>
>   特别说明：
>   1. 50就是conv2的输出通道，相当于这里输出了50个卷积核。
>   2. 4\*4是上一步池化后输出的核的面积。
>
> - fc2也采用线性变换层实现，输入样本大小为500（同fc1输出），输出样本大小为10（十个分类）

- 第二个函数forward指的是前向传播。前向传播指明了传播方向，最后经过log_softmax输出结果。顺序如下：

> - conv1
> - relu
> - max_pool2d
> - conv2
> - relu
> - max_pool2d
> - 一维数据（view函数类似于reshape，是一种矩阵变换，这里变换是因为下面的全连接层只能接受一维数据）
> - fc1
> - relu
> - fc2
> - log_softmax（输出）

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(in_features=4*4*50, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### 对view()函数的测试

使用 torch.arange() 建立一个1D tensor，然后使用view试一试：

In [3]:
a = torch.arange(1, 17)
a

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16])

In [4]:
print(a.view(4, 4))
print(a.view(4, -1))
print(a.view(-1, 4))
print(a.view(2, -1))
print(a.view(-1, 2))

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]])
tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]])
tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]])
tensor([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16]])
tensor([[ 1,  2],
        [ 3,  4],
        [ 5,  6],
        [ 7,  8],
        [ 9, 10],
        [11, 12],
        [13, 14],
        [15, 16]])


可以看出，view(row, col)函数会自动变换tensor的维度，-1就是自动判断行或者列数量

## 加载MNIST数据集

下列加载代码分为三步：
1. 定义数据变换规则
2. 读取MNIST数据集
3. 加载数据

In [13]:
data_tf = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))])

mnist_trainset = datasets.MNIST(root='./MNIST_data/MNIST', train=True, download=True, transform=data_tf)
mnist_testset = datasets.MNIST(root='./MNIST_data/MNIST', train=False, download=True, transform=data_tf)

train_loader = DataLoader(mnist_trainset, batch_size=1000, shuffle=True)
test_loader = DataLoader(mnist_testset, batch_size=1000, shuffle=True)  # 测试集无需打乱

## 训练神经网络

### 加载网络

网络初始化代码如下。其中：优化器为SGD，损失函数为交叉熵损失函数。

In [14]:
net = Net()
if torch.cuda.is_available():  # 如果GPU可以使用
    net = net.cuda(1)
    print("CUDA is available.")
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
loss_func = torch.nn.CrossEntropyLoss()

CUDA is available.


In [15]:
print(net)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


### 开始训练网络

参照第1节代码，5个epoch，每个batch有1000个数据。

>注意区别：
>1. 去掉了CUDA是否可用的判断代码
>2. 这里由于输入后第一层为卷积层，接收的是二维图像数据，所以不用把img变换为一维数据。

In [ ]:
net.train()  # 启用train模式
for epoch in range(5):
    for batch_ndx, data in enumerate(train_loader):  # 按照一个batch = 1000来抽取数据
        img, label = data        
        # 前向传播
        if torch.cuda.is_available():
            img = img.cuda(1)
            label = label.cuda(1)
            # print("使用CUDA训练")
        else:
            pass
        output = net(img)
        loss = loss_func(output, label)
        
        # 反向传播
        optimizer.zero_grad()  # 梯度归零
        loss.backward()  # 损失函数反向传播
        optimizer.step()
        
        if batch_ndx%10 == 0:
            print('epoch: {}, batch_ndx: {}, loss: {:.4}'.format(epoch, batch_ndx, loss.data.item()))  

epoch: 0, batch_ndx: 0, loss: 2.304
epoch: 0, batch_ndx: 10, loss: 2.253
epoch: 0, batch_ndx: 20, loss: 2.205
epoch: 0, batch_ndx: 30, loss: 2.139
epoch: 0, batch_ndx: 40, loss: 2.064
epoch: 0, batch_ndx: 50, loss: 1.95
epoch: 1, batch_ndx: 0, loss: 1.785
epoch: 1, batch_ndx: 10, loss: 1.567
epoch: 1, batch_ndx: 20, loss: 1.364
epoch: 1, batch_ndx: 30, loss: 1.177
epoch: 1, batch_ndx: 40, loss: 0.9952
epoch: 1, batch_ndx: 50, loss: 0.8769
epoch: 2, batch_ndx: 0, loss: 0.779
epoch: 2, batch_ndx: 10, loss: 0.6975
epoch: 2, batch_ndx: 20, loss: 0.6161
epoch: 2, batch_ndx: 30, loss: 0.5728
epoch: 2, batch_ndx: 40, loss: 0.5286
epoch: 2, batch_ndx: 50, loss: 0.5265
epoch: 3, batch_ndx: 0, loss: 0.5426


### 测试网络

代码同第1节。删除了对CUDA的判断及开始的数据一维化。

In [10]:
net.eval()
eval_loss = 0
eval_acc = 0

# 对测试集进行测试
for batch_ndx, data in enumerate(test_loader):
    # 获得img(手写图片)，label标签（手写图片对应数字）
    img, label = data
    if torch.cuda.is_available():
        img = img.cuda(1)
        label = label.cuda(1)
        # print("使用CUDA训练")
    else:
        pass    
    #  向前传播，获得out结果和损失函数
    output = net(img)
    loss = loss_func(output, label)  # 交叉熵
    
    # 损失函数乘标签大小累计
    eval_loss += loss.data.item()*label.size(0)  # 每个计算一次损失？
    # 在10维数据中，获得最大的预测值（即预测数）
    _, pred = torch.max(output, 1)
    # 判断是否与真实结果相同
    num_correct = (pred == label).sum()
    
    # 累计真实结果
    eval_acc += num_correct.item()

print(label.size(0))
# 输出评估结果    
print('Test Loss: {:.6f}, Acc: {:.6f}'.format(
    eval_loss / (len(mnist_testset)),
    eval_acc / (len(mnist_testset))
))

1000
Test Loss: 0.311799, Acc: 0.913100


In [12]:
torch.cuda.empty_cache()